In [21]:
# Libraries and Google Drive
import dataclasses
import typing
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as optimize

# Mount your google drive to get files
from google.colab import drive
#drive.mount('drive')

In [22]:
# Loading from Raw Osciloscope Data

"""
returns (time, data) trmmed so that the first_value and last value are not null_value (first_value can be specified further)
"""
def trim_osc_time_data(time, data, first_value, null_value):
  # i becomes the first index
  i = 0
  if (first_value == -10.16):
    while (data[i] <= null_value):
      i += 1
  else:
    i = 0
    while (i < len(data) and data[i] != first_value):
      i += 1

  # j becomes the last index
  j = len(data) - 1
  while (data[j] <= null_value):
    j -= 1

  data = data[i:j]
  time = time[i:j] - time[i]
  return (time, data)

"""
returns (time, data) of the specified CSV (directly from osciloscope)
"""
def get_osc_time_data(file, first_value = -10.16, null_value = -10.16):
  osc_data = np.genfromtxt(file, delimiter = ',', dtype=None, encoding="utf-8")
  osc_data_arr = [list(i) for i in zip(*osc_data)]
  return trim_osc_time_data(np.array(osc_data_arr[3]), np.array(osc_data_arr[4]), first_value, null_value)

"""
from CHI660E Software
"""
def get_real_data(file):
  raw = np.loadtxt(file, delimiter=",", dtype="float,float")
  return [list(i) for i in zip(*raw)]

# Write in our fit function here
def fit_function(t, a, omega, phi, C):
  return a * np.sin(omega * t + phi) + C

In [23]:
# Fixed Constants for 3,750SPS
alpha = 4194304
beta = 1.8639
IDEAL_OFC = 0
IDEAL_FSC = 4500488
PGA = 16

R = 47000
v_ref = 2.490
DAC_2V = 1.9958
PERIOD = 0.0009999375 # 1 / (3000.1875117194822 / 3)

"""
Returns SIG_OUT Voltage

Vin = SIG_OUT - DAC_2V
"""
def raw_to_sig_out(raw, ofc: int, fsc: int):
  return DAC_2V - (((raw / (fsc * beta)) + (ofc/alpha)) * (2 * v_ref) / PGA)

def raw_to_sig_out_ideal(raw):
  return raw_to_sig_out(raw, IDEAL_OFC, IDEAL_FSC)

"""
From Ohms Law + Transimpedance
"""
def to_current(voltage):
  return -(voltage - DAC_2V) / R

"""
Format:
ofc, fsc
raw
raw
...
"""
def get_voltage(file):
  try:
    f = open(file, 'r').read()
  finally:
    raw = f.split() # by lines
    ofc, fsc = list(map(int, raw[0].split(",")))
    return raw_to_sig_out(np.array(list(map(int, raw[1:]))), ofc, fsc)
  return np.array()


"""
Returns times (calculated from period), voltage
"""
def get_data(file):
  voltage = get_voltage(file)
  return np.arange(0, len(voltage)) * PERIOD, voltage

In [24]:
"""
Dataclasses for voltammetry settings
"""

@dataclasses.dataclass
class VoltammetrySettings:
  startV: float
  endV: float
  incrE: float
  amplitude: float
  sample_width: float

@dataclasses.dataclass
class DPVSettings(VoltammetrySettings):
  pulse_width: float
  pulse_period: float

@dataclasses.dataclass
class SWVSettings(VoltammetrySettings):
  verticesV: typing.List[float]
  frequency: int


"""
Returns voltages (x-axis) and sample_timings (low, high) for that peak
"""
def get_sample_times(setting: VoltammetrySettings):
  # sample timings from https://pineresearch.com/shop/kb/software/methods-and-techniques/voltammetric-methods/differential-pulse-voltammetry-dpv/
  if type(setting) is DPVSettings:
    cycle_count = int((setting.endV - setting.startV) / setting.incrE - 1)
    low_sample = setting.pulse_period - setting.pulse_width - setting.sample_width
    high_sample = setting.pulse_period - setting.sample_width
    voltages = [setting.startV + setting.incrE * i for i in range(cycle_count)] # The peak voltage
    times = [(low_sample + setting.pulse_period * i, high_sample + setting.pulse_period * i) for i in range(cycle_count)]
  elif type(setting) is SWVSettings:
    # TODO
    pass
  else:
    voltages = []
    times = []

  return voltages, times

"""
call by passing in the variables or get_delta_i(*get_data(file), setting, kernel_sizes)

Returns voltages (x-axis) and a two 2D array of currents of len(kernel_sizes) x len(voltages)
"""
def get_high_low_i(volt_time: typing.List[float], volt_cur: typing.List[float], setting: VoltammetrySettings, kernel_sizes: typing.List[int]):
  high_cur = [ [] for _ in range(len(kernel_sizes))]
  low_cur = [ [] for _ in range(len(kernel_sizes))]
  voltages, times = get_sample_times(setting)
  # Find the closest time to the requested sampling time and take the difference
  for low, high in times:
    low_idx = np.searchsorted(volt_time, low, side='left')
    high_idx = np.searchsorted(volt_time, high, side='left')

    for i in range(len(kernel_sizes)):
      size = kernel_sizes[i]
      # +1 because 0 index and it will be on the 'left' of the true value
      high_cur.append(to_current((np.average(volt_cur[high_idx-size+1:high_idx+size]))))
      low_cur.append(to_current((np.average(volt_cur[low_idx-size+1:low_idx+size]))))
  return voltages, high_cur, low_cur

"""
call by passing in the variables or get_delta_i(*get_data(file), setting, kernel_sizes)

Returns voltages (x-axis) and a 2D array of currents of len(kernel_sizes) x len(voltages)
"""
def get_delta_i(volt_time: typing.List[float], volt_cur: typing.List[float], setting: VoltammetrySettings, kernel_sizes: typing.List[int]):
  currents = [ [] for _ in range(len(kernel_sizes))]

  voltages, times = get_sample_times(setting)
  # Find the closest time to the requested sampling time and take the difference
  for low, high in times:
    low_idx = np.searchsorted(volt_time, low, side='left')
    high_idx = np.searchsorted(volt_time, high, side='left')

    for i in range(len(kernel_sizes)):
      size = kernel_sizes[i]
      # +1 because 0 index and it will be on the 'left' of the true value
      currents[i].append(to_current((np.average(volt_cur[high_idx-size+1:high_idx+size])))
                          - to_current((np.average(volt_cur[low_idx-size+1:low_idx+size]))))
  return voltages, currents

In [25]:
def print_noise_example(file, p0 = [to_current(-0.0034), 400, 0, 2.0205]):
  times, voltages = get_data(file)
  current = to_current(voltages[:100])
  param, param_cov = optimize.curve_fit(fit_function, times[:100], current, p0)
  plt.plot(times[:100], fit_function(times[:100], *param), label = "Best Fit Curve")
  plt.scatter(times[:100], current, label = "Device", color="xkcd:violet")
  plt.title("Current vs Time (Noise)")
  plt.xlabel("Time (s)")
  plt.ylabel("Current (A)")
  plt.legend()
  plt.show()
  print("Amplitude: {}±{}, Frequency: {}±{:.2f}".format(param[0], np.sqrt(param_cov[0][0]), param[1] / (2 * np.pi), np.sqrt(param_cov[1][1]) / (2 * np.pi)))

def print_sample_example(file, setting: VoltammetrySettings):
  sample_count = 600
  times, voltages = get_data(file)
  times = times[:sample_count]
  currents = to_current(voltages[:sample_count])

  times = times[300:]
  currents = currents[300:]

  _, sample_times = get_sample_times(setting)
  low_t, high_t = [list(i) for i in zip(*sample_times)]
  low_t = low_t[:np.searchsorted(low_t, times[-1], side='right')]
  high_t = high_t[:np.searchsorted(high_t, times[-1], side='right')]

  plt.plot(times, currents, label = "Device")
  plt.vlines(low_t, min(currents), max(currents), label = "Low Sampling Times", color="xkcd:rose")
  plt.vlines(high_t, min(currents), max(currents), label = "High Sampling Times", color="xkcd:mint green")
  plt.title("Current vs Time (Sampling)")
  plt.xlabel("Time (s)")
  plt.ylabel("Current (A)")
  plt.legend()
  plt.show()

  # close view scatter
  plt.scatter(times[75:100], currents[75:100], label = "Device")
  plt.vlines(low_t, min(currents), max(currents), label = "Low Sampling Times", color="xkcd:rose")
  plt.title("Current vs Time (Low)")
  plt.xlabel("Time (s)")
  plt.ylabel("Current (A)")
  plt.legend()
  plt.show()

  plt.scatter(times[175:200], currents[175:200], label = "Device")
  plt.vlines(high_t, min(currents), max(currents), label = "High Sampling Times", color="xkcd:mint green")
  plt.title("Current vs Time (High)")
  plt.xlabel("Time (s)")
  plt.ylabel("Current (A)")
  plt.legend()
  plt.show()

def print_dpv(file, setting: VoltammetrySettings, kernel_sizes = [2, 4, 8, 16]):
  voltages, currents = get_delta_i(*get_data(file), setting, kernel_sizes)
  for i in range(len(currents)):
    plt.plot(voltages, currents[i], label = "Kernel: {}".format(kernel_sizes[i]))

  real_voltages, real_current = get_real_data("/content/drive/MyDrive/I2BL/Data/Potentiostat/Sample_5_dpv.txt")
  plt.plot(real_voltages, real_current, label = "Potentiostat", color="orange")
  plt.title("Current vs Voltage of Aptamer Sensor")
  plt.xlabel("Voltage (V)")
  plt.ylabel("Current (A)")
  plt.gca().invert_xaxis()
  plt.legend()
  plt.show()

#Potentiostat Usage

1. Run all the functions and link your google drive.
2. Add file path to files list
3. adjust default_setting to the experiment it corresponds to.
4. comment out uncessary print functions
5. run cell.

In [ ]:
files = [
          "/content/drive/MyDrive/I2BL/Data/Potentiostat/PotentioStat_Test_5_PGA_16.txt",
        ]

default_setting = DPVSettings(startV = -0.05, endV = -0.40, incrE = -0.005, amplitude = 0.05, sample_width = 0.0167, pulse_width = 0.1, pulse_period = 0.5)

for file in files:
  print_dpv(file, default_setting)
  #print_noise_example(file)
  print_sample_example(file, default_setting)

In [26]:
"""
Arduino Code:
Serial.print(micros() - start_time); // start_time is set to micros() at beginning of experiment
Serial.print(',');
Serial.println(voltage, 4); // voltage is a float

Format:
time,voltage
time,voltage
...
"""
def get_arduino_dac_data(file):
  arduino_data = np.loadtxt(file, delimiter = ',', dtype='int,float')
  arduino_data_arr = [list(i) for i in zip(*arduino_data)]  # reformat into a 2d array

  # Clean data and put it in the variables time, volt_a, volt_b
  time = np.array(arduino_data_arr[0]) / 10**6 # seconds
  v_a = np.array(arduino_data_arr[1])
  v_b = np.full_like(v_a, 2.0)  # WE always 2.0V
  v_diff = v_a - v_b
  return time, v_a - v_b

"""
file should be the file of csv for the channel measuring DAC_OUT
"""
def get_osc_dac_data(file, start_value = 1.92, null_value = -10.16):
  time, volt_a = get_osc_time_data(file, start_value, null_value)
  volt_b = np.full_like(volt_a, 2.0)
  volt_diff = volt_b - volt_a
  return (time, volt_a, volt_b, volt_diff)


In [27]:
"""
Plot only DAC requested voltages
"""
def plot_voltage(file: str, cut_length: int = 100):
  arduino_time, arduino_volt_a, arduino_volt_b, arduino_diff = get_arduino_dac_data(file)

  # Plot DAC Commands
  plt.step(arduino_time, arduino_volt_a, where="post", label="Refrence (A)")
  plt.step(arduino_time, arduino_volt_b, where="post", label="Working (B)")
  plt.step(arduino_time, arduino_volt_b - arduino_volt_a, where="post", label="Working-Reference (B-A)")
  plt.xlabel("Time (s)")
  plt.ylabel("Voltage (V)")
  plt.title("Voltage vs Time of DAC Commands")
  plt.legend()
  plt.show()

  # WE-RE Only
  plt.step(arduino_time, arduino_volt_b - arduino_volt_a, where="post", color='g', label="Working-Reference (B-A)")
  plt.xlabel("Time (s)")
  plt.ylabel("Voltage (V)")
  plt.title("Working-Refrence vs Time of DAC Commands")
  plt.legend()
  plt.show()

  # Cut Version
  plt.step(arduino_time[1:cut_length], arduino_volt_b[1:cut_length] - arduino_volt_a[1:cut_length], where="post", color='g', label="Working-Refrence (B-A)")
  plt.xlabel("Time (s)")
  plt.ylabel("Voltage (V)")
  plt.title("Working-Refrence vs Time of DAC Commands (Small view)")
  plt.legend()
  plt.show()

"""
call like print_dpv_values(file, cut_time, offset_time)
"""
def print_dpv_values(ar_file, ch_file, cut_time = 0, offset_time = 0):
  time, volt_a, volt_b, volt_diff = get_osc_dac_data(ch_file)
  # Full view
  plt.plot(time, volt_a, label="Reference (A)")
  plt.plot(time, volt_b, label="Working (B)")
  plt.plot(time, volt_diff, label="Working-Reference (B-A)")
  plt.xlabel("Time (s)")
  plt.ylabel("Voltage (V)")
  plt.title("Voltage vs Time of DAC Output")
  plt.legend()
  plt.show()

  # Zoom in
  plt.plot(time, volt_diff, color = 'g', label="Working-Reference (B-A)")
  plt.xlabel("Time (s)")
  plt.ylabel("Voltage (V)")
  plt.title("Working-Reference (B-A) vs Time of DAC Output")
  plt.legend()
  plt.show()

  # Cut Version
  cut_length_osc = 400
  plt.plot(time[:cut_length_osc], volt_diff[:cut_length_osc], color = 'g', label="Working-Reference (B-A)")
  plt.xlabel("Time (s)")
  plt.ylabel("Voltage (V)")
  plt.title("Working-Reference (B-A) vs Time of DAC Output (Small view)")
  plt.legend()
  plt.show()

  # Overlaid
  arduino_time, arduino_volt_a, arduino_volt_b, arduino_diff = get_arduino_dac_data(ar_file)

  # First offset arduino_time

  i = 0
  while (arduino_time[i] < cut_time):
    i += 1

  cut_time = arduino_time[i:] - arduino_time[i] + offset_time
  cut_diff = (arduino_volt_b - arduino_volt_a)[i:]

  # Trim arduino to match osciloscope
  if (cut_time[-1] > time[-1]):
    max_time = time[-1]
    j = len(cut_time) - 1
    while (cut_time[j] > max_time):
      j -= 1
    cut_time = cut_time[:j]
    cut_diff = cut_diff[:j]

  plt.plot(cut_time, cut_diff, color = 'xkcd:cerulean', label="DAC Commands")
  plt.plot(time, volt_diff, color = 'xkcd:orchid', label="Osciloscope")

  plt.xlabel("Time (s)")
  plt.ylabel("Voltage (V)")
  plt.title("Working-Refrence (B-A) vs Time of DAC Commands and Output Overlaid")
  plt.legend()
  plt.show()

  # Cut Overlaid
  if (cut_time[-1] > time[cut_length_osc]):
    max_time = time[cut_length_osc]
    j = len(cut_time) - 1
    while (cut_time[j] > max_time):
      j -= 1
    cut_time = cut_time[:j]
    cut_diff = cut_diff[:j]

  plt.plot(cut_time, cut_diff, color = 'xkcd:cerulean', label="DAC Commands")
  plt.plot(time[:cut_length_osc], volt_diff[:cut_length_osc], color = 'xkcd:orchid', label="Osciloscope")
  plt.xlabel("Time (s)")
  plt.ylabel("Voltage (V)")
  plt.title("Working-Refrence (B-A) vs Time of DAC Commands and Output Overlaid (Small view)")
  plt.legend()
  plt.show()

# DAC Usage

1. Run all the functions and link your google drive.
2. Add tuples of (arduino_file, osc_file) into files
3. run cell.

In [ ]:
files = [ () ]

for ar_file, ch_file in files:
  plot_voltage(ar_file)
  print_dpv_values(ar_file, ch_file)

In [32]:
v_ref = 2.490
R = 47000
IMAX = 20/10**6 # from transimpedance TODO
IMIN = -0.00011

PGAS = [1, 2, 4, 8, 16, 32, 64]
clean_bits =  {
                "1000": [19, 18, 18, 17, 17, 16, 15],
                "2000": [18, 18, 17, 17, 16, 16, 15],
                "3750": [18, 17, 17, 16, 16, 15, 14],
                "7500": [17, 17, 16, 16, 16, 15, 14],
                "15000": [17, 17, 16, 15, 15, 14 ,13],
                "30000": [17, 16, 16, 15, 15, 14, 13],
              }

v_max = np.array([ value for value in ((2 * v_ref) / np.array(PGAS))])
v_min = np.array([ value for value in ((2 * -v_ref) / np.array(PGAS))])
i_max = np.array([min(value, IMAX) for value in (v_max / R)])
i_min = np.array([max(value, IMIN) for value in (v_min / R)])

# 24 bit output t_max-t_min
steps = ((2**24)-1) - (-2**24)

print("Range and resolution based on constants above")

for i in range(len(PGAS)):
  print("\nPGA: {} Min: {:E} Max: {:E}\n".format(PGAS[i], i_min[i], i_max[i]))

  v_step = (v_max[i] - v_min[i]) / steps  # Use v range because ADC raw range is not affected by trans amp
  i_step = v_step / R                     # For single bit but there is noise

  for key in clean_bits.keys():
    lowest_bit_value = 2**(24 - clean_bits[key][i])   # The last x bits are not used
    i_resolution = i_step * lowest_bit_value          # multiply by the lowest_bit_value to resolution
    print("SPS: {} Noise Free Resolution: {:E}".format(key, i_resolution))

Range and resolution based on constants above

PGA: 1 Min: -1.059574E-04 Max: 2.000000E-05

SPS: 1000 Noise Free Resolution: 2.020978E-10
SPS: 2000 Noise Free Resolution: 4.041956E-10
SPS: 3750 Noise Free Resolution: 4.041956E-10
SPS: 7500 Noise Free Resolution: 8.083912E-10
SPS: 15000 Noise Free Resolution: 8.083912E-10
SPS: 30000 Noise Free Resolution: 8.083912E-10

PGA: 2 Min: -5.297872E-05 Max: 2.000000E-05

SPS: 1000 Noise Free Resolution: 2.020978E-10
SPS: 2000 Noise Free Resolution: 2.020978E-10
SPS: 3750 Noise Free Resolution: 4.041956E-10
SPS: 7500 Noise Free Resolution: 4.041956E-10
SPS: 15000 Noise Free Resolution: 4.041956E-10
SPS: 30000 Noise Free Resolution: 8.083912E-10

PGA: 4 Min: -2.648936E-05 Max: 2.000000E-05

SPS: 1000 Noise Free Resolution: 1.010489E-10
SPS: 2000 Noise Free Resolution: 2.020978E-10
SPS: 3750 Noise Free Resolution: 2.020978E-10
SPS: 7500 Noise Free Resolution: 4.041956E-10
SPS: 15000 Noise Free Resolution: 4.041956E-10
SPS: 30000 Noise Free Resolut